# Publish Servable
Send the trained SchNet model to DLHub

In [1]:
from dlhub_sdk.models.servables.python import PythonStaticMethodModel
from dlhub_sdk.utils.types import compose_argument_block
from dlhub_sdk.client import DLHubClient
from dlhub_app import evaluate_molecules
from jcesr_ml.benchmark import load_benchmark_data
from time import sleep
import os

## Start the Servable Description
Load in the function information, and build a basic description

In [2]:
model = PythonStaticMethodModel.from_function_pointer(evaluate_molecules)

Give the model a title and name

In [3]:
model.set_name('g4mp2_delta_schnet').set_title('SchNet Model to Predict G4MP2 Activation Energy from B3LYP Energy and Coordinates')

Describe the function inputs

In [4]:
model.set_inputs('tuple', 'Structures and B3LYP energies of moelcule sto be evaluated',
                 element_types=[
                     compose_argument_block('list', 'Structures of molecules in XYZ format', item_type='string'),
                     compose_argument_block('list', 'B3LYP total energies of Energies of molecules in Ha', item_type='float')
                 ])

In [5]:
model.set_outputs('list', 'Estimate of G4MP2 atomization energy', item_type='float')

## Add Datacite Information
Define who made the model, who should be credited, etc.

In [6]:
model.set_authors(["Ward, Logan", "Blaiszik, Ben", "Foster, Ian", "Assary, Rajeev", "Naranayan, Badri", "Curtis, Larry"],
                 [["Argonne National Laboratory"]]*6)

Added related identifiers

In [7]:
model.add_related_identifier("10.1021/acs.jctc.8b00908", "DOI", "Requires")

Add an abstract

In [8]:
model.set_abstract('This model predicts the G4MP2 atomization energy of a molecule given the relaxed atomic coordinates and total energy from a B3LYP calculation. '
                  'It was trained on a set of 134k organic molecules smaller than 9 heavy atoms (i.e., the GDB9 dataset) plus 66 molecules larger than 9 heavy atoms.')

## Add Relevant Files
We need the model and some utility functions

In [9]:
model.add_file(os.path.join('model', 'architecture.pth'))

In [10]:
model.add_file(os.path.join('model', 'best_model'))

In [11]:
model.add_directory('jcesr_ml', recursive=True, include='*.py')

## Describe Computational Environment
We described our environment using the `repo2docker` configuration files

In [12]:
model.parse_repo2docker_configuration()

In [13]:
model.list_files()

['model/architecture.pth',
 'model/best_model',
 'jcesr_ml/atom_environment.py',
 'jcesr_ml/benchmark.py',
 'jcesr_ml/coordinates.py',
 'jcesr_ml/qml.py',
 'jcesr_ml/schnetpack.py',
 'jcesr_ml/style.py',
 'jcesr_ml/utils.py',
 'jcesr_ml/__init__.py',
 '/mnt/c/Users/ward6/Documents/UC/JCESR/JCESR/all-train-data/schnet-delta/requirements.txt',
 '/mnt/c/Users/ward6/Documents/UC/JCESR/JCESR/all-train-data/schnet-delta/setup.py']

## Publish it to DLHub
Make a client and send the model in

In [14]:
client = DLHubClient(http_timeout=-1)

In [15]:
task_id = client.publish_servable(model)

In [17]:
while client.get_task_status(task_id)['status'] != 'SUCCEEDED':
    sleep(30)

## Make Sure it Works
Send a small number of molecules to DLHub, make sure the model generates a reasonable energy prediction

In [19]:
train_data, test_data = load_benchmark_data()

In [20]:
mols = train_data.sample(9)

In [23]:
client.run(client.get_username() + '/' + model.name, (mols['xyz'].tolist(), mols['u0'].tolist()))

["Internal Error: 'dlhub_1c181fd7_aa56_412c_b389_fc3aec176988'"]